In [3]:
import csv
import cv2
import pytesseract 
from pytesseract import image_to_string
import pandas as pd
import re
import os
import numpy as np
from moviepy.video.io.VideoFileClip import VideoFileClip
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
pytesseract.pytesseract.tesseract_cmd=r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
%run london_olympic_data_helper.ipynb

In [4]:
def save_data(row,fileName):
        # Check if the CSV file exists
    if not os.path.isfile(fileName+".csv"):
        # Create a new CSV file with headers if it doesn't exist
        df = pd.DataFrame(columns=['Sno','Match name','Name','Country','Difficulty','Dive Position','Somersaults', 'Dive Group', 'Twists','Score','Final Score'])
    else:
        # Read the existing CSV file into a DataFrame
        df = pd.read_csv(fileName+".csv")
    
    df
    df = df.append(row, ignore_index=True)
    # Save the updated DataFrame to the CSV file
    df.to_csv(fileName+'.csv', index=False)

    

In [5]:
def correct_score(score):
    pattern = r'(\d+) (\d+(\.\d+)?)?'

    match = re.search(pattern,score)

    if match:
        num2 = match.group(2)
        return num2
    else:
        return score

In [6]:
def extract_from_video(filePath,videoName,template,templateRes):
        # Open the video using OpenCV
    
    desired_width = 1631
    desired_height = 907
    cap = cv2.VideoCapture(filePath)
    #,apiPreference=cv2.CAP_ANY,params=[
    #cv2.CAP_PROP_FRAME_WIDTH,desired_width ,
    #cv2.CAP_PROP_FRAME_HEIGHT, desired_height])
    
    #cap.set(cv2.CAP_PROP_FRAME_WIDTH, desired_width)
    #cap.set(cv2.CAP_PROP_FRAME_HEIGHT, desired_height)
    fps=cap.get(cv2.CAP_PROP_FPS)
    print(fps)
    minToSkip=4
    framesToSkip=fps*60*minToSkip
    framesToSkipEverySec=fps-1
    currentFrame=0
    currentFrameInSec=0
    diveStarted=False
    sno=1
    row=[]

    # Initialize a list to store the scores

    print("start")

    oldText="old"
    # Process each frame of the video
    while True:
        # Read the next frame
        ret, frame = cap.read()
        currentFrame+=1
        currentFrameInSec+=1
        if currentFrame<framesToSkip :
            if currentFrame%(fps*60)==0:
                print(currentFrame//(fps*60) ," min(s) Skipped")
            continue
        if currentFrameInSec<framesToSkipEverySec:
            continue
        
        # Break the loop if the video has ended
        if not ret:
            print("completed")
            break

        current_pos = cap.get(cv2.CAP_PROP_POS_MSEC)
        total_seconds = int(current_pos // 1000)
        minutes = total_seconds // 60
        seconds = total_seconds % 60
        timeInVideo =f"{minutes} minutes and {seconds} seconds"
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Use OCR to extract the text from the frame
        text = image_to_string(gray,lang="eng")   

        words = text.lower().split()  # split the text into a list of words
        filtered_words = [word for word in words if (word != 'olympic' and word != 'olympic' and word!='2012' and word!='201' and word!='london' and word!='london2012' and word!="lon" and word!="londo") ]  # use list comprehension to remove the word "hello"
        text = ' '.join(filtered_words)  # join the remaining words back into a string
        #if(text!=None):
        #    print(text)
        name=""
        round=""
        country=""
        difficulty=""
        divePosition=""
        somersaults=""
        diveGroup=""
        twists=""
    
        
        if not diveStarted and "difficulty" in text.lower() and "penalty" not in text.lower() and "position" in text.lower():
            diveStarted=True
            cropped=extract_frame(frame,template)
            plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
            round,country,name,difficulty,divePosition,somersaults,diveGroup,twists = extract_diveinfo(cropped)
            row = {'Sno':sno,'Match name':videoName,"Time":total_seconds,'Name':name.strip(),'Country':country.strip(),'Difficulty':difficulty,'Dive Position':divePosition,'Somersaults':somersaults,
        'Dive Group':diveGroup,
        'Twists':twists }
            print(timeInVideo," dive started")
      
        if diveStarted and( ("difficulty" in text.lower() and "penalty" in text.lower()) | ("penalty" in text.lower()) ):
            print(timeInVideo," dive scores announced")
            cropped=extract_frame(frame,templateRes)
            plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
            croppedText=extract_text(cropped)
            score=get_score(cropped)
            finalScore=get_final_score(cropped,text).strip()
            finalScore=correct_score(finalScore)
            #print(finalScore,score)
            row['Score'] = score
            row['Final Score'] = finalScore
            extract_clip(filePath,'ExtractedVideos/'+str(sno)+'_'+videoName+'.mp4',int(row['Time']),int(total_seconds))
            diveStarted=False
            sno+=1
            #print(text)
            print(row)
            save_data(row,"scores")

        

        currentFrameInSec=0    
    cap.release()

In [7]:
templateLondon = cv2.imread('HelperImages/template_london.png', cv2.IMREAD_GRAYSCALE)
templateResLondon = cv2.imread('HelperImages/template_res_london.png', cv2.IMREAD_GRAYSCALE)

extract_from_video("DownloadedVideos/Men_10m_Prel_London2012.mp4","Men_10m_Prel_London2012",templateLondon,templateResLondon)

0.0
start


ZeroDivisionError: float floor division by zero